# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Working with collections in F#”

Isaac Abraham proposes three types of C# developers working with collections:

1. _The C# 2 developer:_ the one who is doggedly imperative and is well within the OS-level shadow of C++, staying firmly with `for`, `while` and `do` loops.
2. _The LINQ developer:_ the one who is quite fond of using SQL set operations in C# with naïve disregard for expensive allocations and mutations.
3. _The wannabe FP developer:_ the one who understands the complexity-control benefits of the functional approach and might even dabble in the [immutable collections](https://docs.microsoft.com/en-us/archive/msdn-magazine/2017/march/net-framework-immutable-collections) of C#.

Two out of three of these types might find the following challenging:

>Given a set of football results …which teams won the most away games in the season.

In [ ]:
type FootballResult = { HomeTeam : string; AwayTeam : string; HomeGoals : int; AwayGoals : int }

let create (ht, hg) (at, ag) =
    { HomeTeam = ht; AwayTeam = at; HomeGoals = hg; AwayGoals = ag }


The `create` function is a brilliant mix: a curried function taking tuple arguments! We use this function to generate a set of data such that the answer to our football question is:

- Bale Town: 2 wins
- Ronaldo City: 1 win

In [ ]:
let results =
    [
        create ("Messiville", 1) ("Ronaldo City", 2)
        create ("Messiville", 1) ("Bale Town", 3)
        create ("Bale Town", 3) ("Ronaldo City", 1)
        create ("Bale Town", 2) ("Messiville", 1)
        create ("Ronaldo City", 4) ("Messiville", 2)
        create ("Ronaldo City", 1) ("Bale Town", 2)
    ]

I will resort to LINQ, approaching this challenge, reminding myself that lesson 13 introduces the use of inline functions (lambda syntax):

In [ ]:
open System.Linq

let win result =
    if result.HomeGoals > result.AwayGoals then 0
    else 1

results
    .GroupBy(fun result -> result.AwayTeam)
    .Select(fun group -> (group.First().AwayTeam, group.Sum(fun result -> win result)))
    .OrderByDescending(fun tuple -> snd tuple)
    .Select(fun tuple -> $"{fst tuple}: {snd tuple} wins")

index,value
0,Bale Town: 2 wins
1,Ronaldo City: 1 wins
2,Messiville: 0 wins


In C#, I would have used [anonymous objects](https://docs.microsoft.com/en-us/dotnet/csharp/fundamentals/types/anonymous-types) instead of a tuple to get the results. The F# equivalent of C# anonymous objects are _object expressions_ [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/object-expressions)] which are not mentioned until lesson 25—using them above would be skipping ahead too far in this study!

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about